In [1]:
from monomials import Monomial
from polynomials import PolynomialRing, Polynomial

def pad(m, n, zero):
    # makes two lists the same size by appending zero
    while len(m) < len(n):
        m.append(zero)
    while len(n) < len(m):
        n.append(zero)
    
def lt_divides(g, f):
    # determine whether the leading term of g divides that of f
    m = g._leading_monomial().degrees
    n = f._leading_monomial().degrees
    pad(m, n, f.field.zero())
    
    lt = [m[i] <= n[i] for i in range(len(m))]
    return all(lt)

def divide_lt(g, f):
    # actually divides the leading term of f by that of g
    m = g._leading_monomial().degrees
    n = f._leading_monomial().degrees
    pad(m, n, f.field.zero())
    
    a = f.coefs[f._leading_monomial().to_idx()]
    b = g.coefs[g._leading_monomial().to_idx()]
    
    divided = [n[i] - m[i] for i in range(len(m))]
    monomial_idx = Monomial(divided, f.order).to_idx()
    
    coefs = [f.field.zero() for _ in range(monomial_idx)] + [a / b]
    return Polynomial(coefs, f.ring)

def divide(f, g):
    # performs the division algorithm of f by g
    # Assumes that the inputs are okay
    one = f.ring.one()
    zero = f.ring.zero()
    
    q, r = zero, f
    while r != zero and lt_divides(g, r):
        q = q + divide_lt(g, r)
        r = r - divide_lt(g, r)*g
    return q,r    

### Division algorithm

In [2]:
R = PolynomialRing(labels=['x'], base_field='QQ')
x = R.get_vars()[0]

In [3]:
f = x**3+2*x**2+x+1
g = 2*x+1

In [4]:
q, r = divide(f,g)
print(f'{f} is ({q}) * ({g}) + {r}')

x^3 + 2x^2 + x + 1 is (1/2x^2 + 3/4x + 1/8) * (2x + 1) + 7/8


In [5]:
q * g + r == f

True

### GCD via the Euclidean algo

In [6]:
def gcd(f, g):
    if f._leading_monomial() < g._leading_monomial():
        f, g = g, f
    h = f.copy()
    s = g.copy()
    zero = f.ring.zero()
    while s != zero:
        _, rem = divide(h, s)
        h = s
        s = rem
    
    # make leading coefficient one
    c = h.coefs[h._leading_monomial().to_idx()]

    return c**(-1)*h

In [7]:
gcd(x**4-1, x**6-1)

x^2 + -1

In [8]:
def gcd_list(lst):
    l = lst.copy()
    while len(l) > 1:
        g = gcd(l[-1], l[-2])
        l = l[:-2] + [g]
    return l[0]

In [9]:
gcd_list([x**3-3*x+2, x**4-1, x**6-1])

x + -1

### Ideal membership
Then if we want ideal membership tests, we just do the following: we want to answer the question "is $x^3+4x^2+3x-7$ in the ideal generated by $x^3-3x+2$, $x^4-1$, and $x^6-1$?"

In [10]:
# get the ideal generator
gen = gcd_list([x**3-3*x+2, x**4-1, x**6-1])

In [11]:
print(gen)

x + -1


In [12]:
# Then do the division algorithm
q, r = divide(x**3+4*x**2+3*x-7, gen)
print(r)

1


Since the remainder is $1\ne 0$, this proves that the polynomial is not in our ideal.

Another example (problem 9 in section 5 of IVT):

In [13]:
gen = gcd_list([x**3+x**2-4*x-4, x**3-x**2-4*x+4, x**3-2*x**2-x+2])

In [14]:
gen

x + -2

In [15]:
q,r = divide(x**2-4, gen)

In [16]:
r == R.zero()

True

In [17]:
S = PolynomialRing(labels=['x','y','z'])
o = S.ordering

def monomial_derivative(mon, ring):
    degrees = mon.degrees
    idxs = []
    max_idx = 0
    for i in range(len(degrees)):
        if degrees[i] >= 1:
            new_degrees = degrees.copy()
            new_degrees[i] -= 1
            m = Monomial(new_degrees, mon.order)
            idx = m.to_idx()
            if idx > max_idx:
                max_idx = idx
            idxs.append((idx, ring.field.one() * degrees[i]))
    coefs = [ring.field.zero()]*(max_idx + 1)
    for idx, coef in idxs:
        coefs[idx] = coef
    return Polynomial(coefs, ring)

In [18]:
m = Monomial([1,1,1], S.ordering)
print(m)

xyz


In [19]:
monomial_derivative(m, S)

xy + xz + yz

In [20]:
def derivative(f):
    ring = f.ring
    
    result = f.ring.zero()
    
    for i, coef in enumerate(f.coefs):
        derivative = monomial_derivative(f.order.idx_to_monomial(i), ring)
        result += coef * derivative
    
    return result

In [21]:
x,y,z = S.get_vars()
derivative(x**2*y-2*x*y+z**8)

8z^7 + x^2 + 2xy + -2x + -2y

The derivative above works for arbitrary polynomial rings, but we want to switch back to $k[x]$ for this demonstration of the principle in problem 15:

In [22]:
x = R.get_vars()[0]

In [23]:
def square_free(f):
    g = gcd(f, derivative(f))
    q, r = divide(f, g)
    assert r == f.ring.zero()
    return q

In [24]:
square_free(x**(11)-x**10+2*x**8-4*x**7+3*x**5-3*x**4+x**3+3*x**2-x-1)

x^5 + x^2 + -1x + -1